In [1]:
# Libraries required along the code
import numpy as np
np.set_printoptions(threshold = np.inf)
#from SPI_MASTER import *

In [2]:
#===========================================================================
# COMPLEMENTARY Function
#===========================================================================
# Function to emulate binary add operation in 8 size vector
def ADD_BIN (vector):
    x = np.uint8(1)
    # Vector is passed to decimal scalar
    decV = np.array([vector[i]*2**(np.size(vector) - i - 1) for i in range(np.size(vector))])
    add = sum(decV) + 1
    # The reverse process is done and passed to binary vector
    binV = np.array([1 if add & (1 << (np.size(vector) - j - 1)) else 0 for j in range(np.size(vector))],dtype = 'uint8')
    
    return binV

In [106]:
#===========================================================================
# MODULE Definition
#===========================================================================
# Function describing SPI serial communication protocol for slave peripherals
def SPI_MASTER (SPI_MASTER_CLOCK_50,SPI_MASTER_RESET_InHigh,SPI_MASTER_MISO_In,SPI_MASTER_start_InHigh,
               SPI_MASTER_data_In,CLOCK_RATIO,DATAWIDTH_BUS,STATE_SIZE):
    
    #===========================================================================
    # PARAMETER Declarations
    #===========================================================================
    #//////////// STATES ////////////
    State_IDLE = np.uint8(0)
    State_WAIT_HALF = np.uint8(1)
    State_TRANSFER = np.uint8(2)
    State_MOSI = np.uint8(3)
    State_READ_DATA = np.uint8(4)
    State_FULL = np.uint8(5)
    State_END = np.uint8(6)
    #//////////// SIZES ////////////
    
    #===========================================================================
    # PORT Declarations
    #===========================================================================
    #//////////// OUTPUTS ////////////
    # Default values
    SPI_MASTER_MOSI_Out = np.uint8(1)
    SPI_MASTER_newData_Out = np.uint8(0)
    SPI_MASTER_busy_Out = np.uint8(0)
    SPI_MASTER_SCK_Out = np.uint8(0)
    SPI_MASTER_data_Out = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')
    
    #===========================================================================
    # REG/WIRE Declarations
    #===========================================================================
    # Registers (Q)
    global MOSI_Register,NewData_Register,Busy_Register,SCK_Register,DataOut_Register
    global State_Register,MISO_Register,Start_Register,Data_Register,BitCounter_Register
    
    #===========================================================================
    # STRUCTURAL Coding
    #===========================================================================
    # INPUT LOGIC: Combinational
    # Signals (D)
    MISO_Signal = SPI_MASTER_MISO_In
    Start_Signal = SPI_MASTER_start_InHigh
    Data_Signal = Data_Register
    BitCounter_Signal = BitCounter_Register
    
    if State_Register == State_IDLE:
        BitCounter_Signal = 0
        if Start_Register:
            State_Signal = State_WAIT_HALF
        else:
            State_Signal = State_IDLE

    elif State_Register == State_WAIT_HALF:
        Data_Signal = SPI_MASTER_data_In
        if np.array_equal(SCK_Register[1:],np.ones(CLOCK_RATIO - 1,dtype = 'uint8')) and SCK_Register[0] == np.uint8(0):
            State_Signal = State_TRANSFER
        else:
            State_Signal = State_WAIT_HALF
        
    elif State_Register == State_TRANSFER:
        if np.array_equal(SCK_Register,np.zeros(CLOCK_RATIO,dtype = 'uint8')):
            State_Signal = State_MOSI
        elif np.array_equal(SCK_Register[1:],np.ones(CLOCK_RATIO - 1,dtype = 'uint8')) and SCK_Register[0] == np.uint8(0):
            State_Signal = State_READ_DATA
        elif np.array_equal(SCK_Register,np.ones(CLOCK_RATIO,dtype = 'uint8')):
            State_Signal = State_FULL
        else:
            State_Signal = State_TRANSFER
                
    elif State_Register == State_MOSI:
        State_Signal = State_TRANSFER

    elif State_Register == State_READ_DATA:
        State_Signal = State_TRANSFER
        Data_Signal = np.array([Data_Register[k] for k in range(0,7)])
        Data_Signal = np.insert(Data_Signal,0,MISO_Register,axis = 0)
        
    elif State_Register == State_FULL:
        BitCounter_Signal = BitCounter_Register + 1;
        if BitCounter_Register == DATAWIDTH_BUS-1:
            State_Signal = State_END
        else:
            State_Signal = State_TRANSFER
        
    elif State_Register == State_END:
        State_Signal = State_IDLE
        BitCounter_Signal = 0
        
    else:
        State_Signal = State_IDLE
    
    #===========================================================================
    # OUTPUTS
    #===========================================================================
    # OUTPUT LOGIC: Combinational
    # Signals (D)
    MOSI_Signal = MOSI_Register
    NewData_Signal = NewData_Register
    Busy_Signal = Busy_Register
    SCK_Signal = SCK_Register
    DataOut_Signal = DataOut_Register
    
    if State_Register == State_IDLE:
        MOSI_Signal = np.uint8(1)
        NewData_Signal = np.uint8(0)
        Busy_Signal = np.uint8(0)
        SCK_Signal = np.zeros(CLOCK_RATIO,dtype = 'uint8')
        DataOut_Signal = DataOut_Register

    elif State_Register == State_WAIT_HALF:
        MOSI_Signal = MOSI_Register
        NewData_Signal = np.uint8(0)
        Busy_Signal = np.uint8(0)
        SCK_Signal = ADD_BIN(SCK_Register)
        if np.array_equal(SCK_Register[1:],np.ones(CLOCK_RATIO - 1,dtype = 'uint8')) and SCK_Register[0] == np.uint8(0):
            SCK_Signal = np.zeros(CLOCK_RATIO,dtype = 'uint8')
        DataOut_Signal = DataOut_Register
        
    elif State_Register == State_TRANSFER:
        MOSI_Signal = MOSI_Register
        NewData_Signal = np.uint8(0)
        Busy_Signal = np.uint8(1)
        SCK_Signal = ADD_BIN(SCK_Register)
        DataOut_Signal = DataOut_Register
        
    elif State_Register == State_MOSI:
        MOSI_Signal = Data_Register[7]
        NewData_Signal = np.uint8(0)
        Busy_Signal = np.uint8(1)
        SCK_Signal = ADD_BIN(SCK_Register)
        DataOut_Signal = DataOut_Register

    elif State_Register == State_READ_DATA:
        MOSI_Signal = MOSI_Register
        NewData_Signal = np.uint8(0)
        Busy_Signal = np.uint8(1)
        SCK_Signal = ADD_BIN(SCK_Register)
        DataOut_Signal = DataOut_Register
        
    elif State_Register == State_FULL:
        MOSI_Signal = MOSI_Register
        NewData_Signal =  np.uint8(0)
        Busy_Signal =  np.uint8(0)
        SCK_Signal = SCK_Register
        DataOut_Signal = DataOut_Register
        
    elif State_Register == State_END:
        MOSI_Signal = MOSI_Register
        NewData_Signal = np.uint8(1)
        Busy_Signal = np.uint8(0)
        SCK_Signal = SCK_Register
        DataOut_Signal = Data_Register

    else:
        MOSI_Signal = np.uint8(1)
        NewData_Signal = np.uint8(0)
        Busy_Signal = np.uint8(0)
        SCK_Signal = np.zeros(CLOCK_RATIO,dtype = 'uint8')
        DataOut_Signal = DataOut_Register
        
    # STATE REGISTER : Sequential
    if SPI_MASTER_CLOCK_50:
        # Updating global registers
        if SPI_MASTER_RESET_InHigh:
            MOSI_Register = np.uint8(1)
            NewData_Register = np.uint8(0)
            Busy_Register = np.uint8(0)
            SCK_Register = np.zeros(CLOCK_RATIO,dtype = 'uint8')
            DataOut_Register = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')
            State_Register = State_IDLE
            MISO_Register = np.uint8(1)
            Start_Register = np.uint8(0)
            Data_Register = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')
            BitCounter_Register = 0
        else:
            MOSI_Register = MOSI_Signal
            NewData_Register = NewData_Signal
            Busy_Register = Busy_Signal
            SCK_Register = SCK_Signal
            DataOut_Register = DataOut_Signal
            State_Register = State_Signal
            MISO_Register = MISO_Signal
            Start_Register = Start_Signal
            Data_Register = Data_Signal
            BitCounter_Register = BitCounter_Signal
            
    # OUTPUT ASSIGNMENTS
    SPI_MASTER_MOSI_Out = MOSI_Register
    SPI_MASTER_newData_Out = NewData_Register
    SPI_MASTER_busy_Out = Busy_Register
    SPI_MASTER_SCK_Out = (int(not((not SCK_Register[0]) and ((State_Register == State_TRANSFER) or 
                                                             (State_Register == State_MOSI) or                                                             
                                                             (State_Register == State_READ_DATA) or
                                                             (State_Register == State_FULL) or
                                                             (State_Register == State_END)))))
    SPI_MASTER_data_Out = DataOut_Register
    
    return SPI_MASTER_MOSI_Out,SPI_MASTER_newData_Out,SPI_MASTER_busy_Out,SPI_MASTER_SCK_Out,SPI_MASTER_data_Out

In [105]:
#===========================================================================
# MODULE Definition
#===========================================================================
# Function in charge of the input and output signals (I/O)
def BB_SYSTEM (BB_SYSTEM_CLOCK_50,BB_SYSTEM_RESET_InHigh,BB_SYSTEM_MISO_In,BB_SYSTEM_start_InHigh,
               BB_SYSTEM_data_In):
    #===========================================================================
    # PARAMETER Declarations
    #===========================================================================
    # Data width of the imput bus
    DATAWIDTH_BUS = 8
    # Size for the states needed into the protocol
    STATE_SIZE = 3
    # Value to specify how much be the clock for synchronizing Master and Slave
    CLOCK_RATIO = 4

    #===========================================================================
    # PORT Declarations
    #===========================================================================
    #//////////// OUTPUTS ////////////
    # Default values
    BB_SYSTEM_MOSI_Out = np.uint8(1)
    BB_SYSTEM_newData_Out = np.uint8(0)
    BB_SYSTEM_busy_Out = np.uint8(0)
    BB_SYSTEM_SCK_Out = np.uint8(0)
    BB_SYSTEM_data_Out = np.zeros(DATAWIDTH_BUS,dtype = 'uint8')
    
    #===========================================================================
    # REG/WIRE Declarations
    #===========================================================================
    
    #===========================================================================
    # STRUCTURAL Coding
    #===========================================================================
    # Port map - connection between master ports and signals/registers  
    (BB_SYSTEM_MOSI_Out,BB_SYSTEM_newData_Out,BB_SYSTEM_busy_Out,
     BB_SYSTEM_SCK_Out,BB_SYSTEM_data_Out) = SPI_MASTER (BB_SYSTEM_CLOCK_50,BB_SYSTEM_RESET_InHigh,BB_SYSTEM_MISO_In,
                                                        BB_SYSTEM_start_InHigh,BB_SYSTEM_data_In,
                                                        CLOCK_RATIO,DATAWIDTH_BUS,STATE_SIZE)
    
    # OUTPUT ASSIGNMENTS
    return BB_SYSTEM_MOSI_Out,BB_SYSTEM_newData_Out,BB_SYSTEM_busy_Out,BB_SYSTEM_SCK_Out,BB_SYSTEM_data_Out

In [104]:
#===========================================================================
# SIMULATION Coding
#===========================================================================
# TB_SYSTEM
# Files to build the test vectors are opened and readed
fMaster = "prueba.txt"
fileMaster = np.fromfile(fMaster,dtype = 'uint8')
fSlave = "miso_Out.txt"
MISO_In = np.loadtxt(fSlave,dtype = 'uint8')

# Parameters to describe the size of the arrays
# CLOCK_RATIO = 1/16th (2^4 = 16 CLOCK cycles) 
CLOCK_RATIO = 4
FREQUENCY = 50e6

# The variables to treat the file are created
nBits = 8
data_In = np.zeros((nBits,np.size(fileMaster)),dtype = 'uint8')

# The file with integers array is converted into bits array of type unsigned integers of size 8
for r in range(np.size(fileMaster)):
    for s in range(nBits):
        if fileMaster[r] & (1 << (7 - s)):
            bit = 1
        else:
            bit = 0
        # The vector is given according to the protocol as LSB first
        data_In[s,r] = bit

# Variable to emulate the 50MHz clock        
CLOCK_50 = 0

# The test vectors to use for the communication protocol are created
nSim = 1
resetV = np.zeros(nSim,dtype = 'uint8')
resetV[0] = np.uint8(0)
startV = np.zeros(nSim,dtype = 'uint8')
startV[0] = np.uint8(1)

# Arrays composed of protocol outputs in each clock cycle
MOSI_OutV = np.zeros(np.size(MISO_In),dtype = 'uint8')
newData_OutV = np.zeros(np.size(MISO_In) + (2 + nBits*3)*np.size(fileMaster),dtype = 'uint8')
busy_OutV = np.zeros(np.size(MISO_In) + (2 + nBits*3)*np.size(fileMaster),dtype = 'uint8')
SCK_OutV = np.zeros(np.size(MISO_In),dtype = 'uint8')
data_OutV = np.zeros((nBits,np.size(fileMaster)),dtype = 'uint8')

# The communication protocol is tested for nSim simulations
h = 0
while h < nSim:
    i = 0
    j = 0
    k = 0
    
    # The communication protocol registers are defined on its initial values
    MOSI_Register = np.uint8(1)
    NewData_Register = np.uint8(0)
    Busy_Register = np.uint8(0)
    SCK_Register = np.zeros(CLOCK_RATIO,dtype = 'uint8')
    DataOut_Register = np.zeros(nBits,dtype = 'uint8')
    State_Register = np.uint8(0)
    MISO_Register = np.uint8(1)
    Start_Register = np.uint8(0)
    Data_Register = np.zeros(nBits,dtype = 'uint8')
    BitCounter_Register = 0
    while j < np.size(MISO_In):
        CLOCK_50 = np.uint8(0) if CLOCK_50 else np.uint8(1)
        MOSI_Out,newData_Out,busy_Out,SCK_Out,data_Out = BB_SYSTEM (CLOCK_50,resetV[h],MISO_In[j],startV[h],data_In[:,k])
        if CLOCK_50:
            newData_OutV[i] = newData_Out
            busy_OutV[i] = busy_Out
            if newData_Out:
                data_OutV[:,k] = data_Out
                #print('Out: ',data_OutV[:,k])
                #print('In:  ',data_In[:,k])

                k = k + 1
                
            if busy_Out:
                MOSI_OutV[j] = MOSI_Out 
                SCK_OutV[j] = SCK_Out
                j = j + 1
                
            i = i + 1
    h = h + 1

# Out to the receptor MISO input
np.savetxt('mosi_Out.txt',MOSI_OutV,fmt = '%u')
np.savetxt('sck_Out.txt',SCK_OutV,fmt = '%u')
print(MOSI_OutV,'\t',SCK_OutV)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 